In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint

sys.path.append(str(Path("../").resolve()))
pprint(sys.path)

import df_file_interchange as fi


['/home/peterma/exio_dev/repo/df_file_interchange/scratch',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python311.zip',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11/lib-dynload',
 '',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11/site-packages',
 '/home/peterma/exio_dev/repo/df_file_interchange']


In [2]:
unit_cur_a = fi.ci.unit.currency.FICurrencyUnit(unit_desc="USD", unit_multiplier=1000)
unit_cur_b = fi.ci.unit.currency.FICurrencyUnit(unit_desc="EUR", unit_multiplier=1000)
unit_cur_c = fi.ci.unit.currency.FICurrencyUnit(unit_desc="JPY", unit_multiplier=1000000)
unit_cur_d = fi.ci.unit.currency.FICurrencyUnit(unit_desc="USD", unit_multiplier=1000)

extra_info = fi.ci.structured.FIStdExtraInfo(author="shug")

In [3]:
cust_info = fi.ci.structured.FIStructuredCustomInfo(extra_info=extra_info, col_units={"a": unit_cur_a, "b": unit_cur_b, "c": unit_cur_c, "d": unit_cur_d})

ValidationInfo(config={'title': 'FIStructuredCustomInfo'}, context=None, data={'unstructured_data': {}}, field_name='extra_info')
ValidationInfo(config={'title': 'FIStructuredCustomInfo'}, context=None, data={'unstructured_data': {}, 'extra_info': FIStdExtraInfo(author='shug', classname='FIStdExtraInfo')}, field_name='col_units')


In [4]:
serialized_cust_info = cust_info.model_dump()
serialized_cust_info

{'unstructured_data': {},
 'extra_info': {'author': 'shug', 'classname': 'FIStdExtraInfo'},
 'col_units': {'a': {'unit_desc': 'USD',
   'unit_multiplier': 1000.0,
   'classname': 'FICurrencyUnit'},
  'b': {'unit_desc': 'EUR',
   'unit_multiplier': 1000.0,
   'classname': 'FICurrencyUnit'},
  'c': {'unit_desc': 'JPY',
   'unit_multiplier': 1000000.0,
   'classname': 'FICurrencyUnit'},
  'd': {'unit_desc': 'USD',
   'unit_multiplier': 1000.0,
   'classname': 'FICurrencyUnit'}}}

In [5]:
# del serialized_cust_info["col_units"]
# new_cust_info = fi.ci.structured.FIStructuredCustomInfo(**serialized_cust_info)
ctx_cust_info = {
    "clss_extra_info": {
        "FIStdExtraInfo": fi.ci.structured.FIStdExtraInfo,
    },
    "clss_col_units": {
        "FIBaseUnit": fi.ci.unit.base.FIBaseUnit,
        "FICurrencyUnit": fi.ci.unit.currency.FICurrencyUnit,
    },
}


new_cust_info = fi.ci.structured.FIStructuredCustomInfo.model_validate(serialized_cust_info, context=ctx_cust_info)

# with fi.file.rw.init_context(ctx_cust_info):
#     new_cust_info = fi.ci.structured.FIStructuredCustomInfo(**serialized_cust_info)

ValidationInfo(config={'title': 'FIStructuredCustomInfo'}, context={'clss_extra_info': {'FIStdExtraInfo': <class 'df_file_interchange.ci.structured.FIStdExtraInfo'>}, 'clss_col_units': {'FIBaseUnit': <class 'df_file_interchange.ci.unit.base.FIBaseUnit'>, 'FICurrencyUnit': <class 'df_file_interchange.ci.unit.currency.FICurrencyUnit'>}}, data={'unstructured_data': {}}, field_name='extra_info')
ValidationInfo(config={'title': 'FIStructuredCustomInfo'}, context={'clss_extra_info': {'FIStdExtraInfo': <class 'df_file_interchange.ci.structured.FIStdExtraInfo'>}, 'clss_col_units': {'FIBaseUnit': <class 'df_file_interchange.ci.unit.base.FIBaseUnit'>, 'FICurrencyUnit': <class 'df_file_interchange.ci.unit.currency.FICurrencyUnit'>}}, data={'unstructured_data': {}, 'extra_info': FIStdExtraInfo(author='shug', classname='FIStdExtraInfo')}, field_name='col_units')


In [6]:
new_cust_info

FIStructuredCustomInfo(unstructured_data={}, extra_info=FIStdExtraInfo(author='shug', classname='FIStdExtraInfo'), col_units={'a': FICurrencyUnit(unit_desc='USD', unit_multiplier=1000.0, classname='FICurrencyUnit'), 'b': FICurrencyUnit(unit_desc='EUR', unit_multiplier=1000.0, classname='FICurrencyUnit'), 'c': FICurrencyUnit(unit_desc='JPY', unit_multiplier=1000000.0, classname='FICurrencyUnit'), 'd': FICurrencyUnit(unit_desc='USD', unit_multiplier=1000.0, classname='FICurrencyUnit')})

In [ ]:
new_cust_info.model_dump()

{'unstructured_data': {},
 'extra_info': {'author': 'shug', 'classname': 'FIStdExtraInfo'},
 'col_units': {'a': {'unit_desc': 'USD',
   'unit_multiplier': 1000.0,
   'classname': 'FICurrencyUnit'},
  'b': {'unit_desc': 'EUR',
   'unit_multiplier': 1000.0,
   'classname': 'FICurrencyUnit'},
  'c': {'unit_desc': 'JPY',
   'unit_multiplier': 1000000.0,
   'classname': 'FICurrencyUnit'},
  'd': {'unit_desc': 'USD',
   'unit_multiplier': 1000.0,
   'classname': 'FICurrencyUnit'}}}

In [ ]:
from pydantic import BaseModel, ValidationInfo, field_validator

class Model(BaseModel):
    text: str

    @field_validator('text')
    @classmethod
    def remove_stopwords(cls, v: str, info: ValidationInfo):
        context = info.context
        if context:
            stopwords = context.get('stopwords', set())
            v = ' '.join(w for w in v.split() if w.lower() not in stopwords)
        return v


data = {'text': 'This is an example document'}
print(Model.model_validate(data))  # no context
#> text='This is an example document'
print(Model.model_validate(data, context={'stopwords': ['this', 'is', 'an']}))

text='This is an example document'
text='example document'
